In [1]:
# coding:utf-8
"""
本篇为user_all(),从数据库中读取所有暂存的中间表，连接所有分表，得到用户基础信息的宽表
"""
import pandas as pd
import numpy as np
from impala.dbapi import connect
from impala.util import as_pandas
import cPickle
import re
import datetime as dt
import pymysql.cursors

# Global variables
mysql_host = '172.21.1.151'
mysql_port = 3306
mysql_user = 'xli'
mysql_password = '123456'
mysql_db = 'kdwtemp2'
mysql_charset = 'utf8mb4'

In [19]:
#     # 1. 读取user_info结果表
#     sql = 'SELECT * FROM ljl_user_info'
#     user_info = pd.read_sql(sql,conn)
    
#     # 2. 读取get_coupon
#     sql = 'SELECT * FROM ljl_get_coupon'
#     get_coupon = pd.read_sql(sql,conn)
#     df = pd.merge(get_coupon,user_info,how='left',left_on='customerid',right_on='customerid')
    
#     conn.close()
#     return df

#     # 3.读取use_coupon
#     sql = 'SELECT * FROM ljl_use_coupon'
#     use_coupon = pd.read_sql(sql,conn)
    
#     # 4. 读取destine
#     sql = 'SELECT * FROM ljl_destine'
#     destine = pd.read_sql(sql,conn)
    
#     # 5. 读取tag_age_from_song
#     sql = 'SELECT * FROM ljl_tag_age_from_song'
#     tag_age_from_song = pd.read_sql(sql,conn)
    
#     # 6. 读取shop
#     sql = 'SELECT * FROM ljl_shop'
#     shop = pd.read_sql(sql,conn)
    

# 1. 读取user_info结果表

In [2]:
#def user_all():
# 连接数据库以读取中间表
conn = pymysql.connect(host=mysql_host,
                                  port=mysql_port,
                                  user=mysql_user,
                                  password=mysql_password,
                                  db=mysql_db,
                                  charset=mysql_charset)
#     cur = conn.cursor()
#     sql = """CREATE TABLE ljl_all_user as（SELECT ljl_user_info.*,ljl_get_coupon.coupon_get_cnt,ljl_get_coupon.coupon_get_sum,ljl_get_coupon.source_company_id,
#     ljl_get_coupon.source_company_cnt,ljl_get_coupon.get_coupon_type_cnt,ljl_get_coupon.get_com_roomnum,ljl_get_coupon.get_com_regioncode,
#     ljl_get_coupon.get_com_managetype,ljl_get_coupon.get_com_vodsystemtype,ljl_get_coupon.get_coupon_crosscom
#             FROM ljl_user_info LEFT OUTER JOIN ljl_get_coupon
#             on ljl_user_info.customerid=ljl_get_coupon.customerid）"""
#     cur.execute(sql)
#     cur.close()
# 1. 读取user_info结果表
sql = 'SELECT * FROM ljl_user_info'
user_info = pd.read_sql(sql,conn)

#     # 2. 读取get_coupon
#     sql = 'SELECT * FROM ljl_get_coupon'
#     get_coupon = pd.read_sql(sql,conn)
#     df = pd.merge(get_coupon,user_info,how='left',left_on='customerid',right_on='customerid')
# conn.close()
# return user_info

In [3]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 475776 entries, 0 to 475775
Data columns (total 20 columns):
customerid                      475776 non-null object
registertime                    368275 non-null object
customer_status                 368275 non-null object
openid                          195848 non-null object
wechat_sex                      81738 non-null object
wechat_city                     81738 non-null object
wechat_province                 81738 non-null object
wechat_country                  81738 non-null object
kmid                            107657 non-null object
member_birthday                 15559 non-null float64
km_sex                          15559 non-null float64
accountcash                     15559 non-null object
accountbalance                  15559 non-null object
integralbalance                 15559 non-null object
consumetotal                    15559 non-null object
consumetimes                    15559 non-null object
member_age_cut    

# 2. 和get_coupon merge

In [4]:
# 2. 读取get_coupon
sql = 'SELECT * FROM ljl_get_coupon'
get_coupon = pd.read_sql(sql,conn)

In [5]:
get_coupon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145904 entries, 0 to 145903
Data columns (total 11 columns):
customerid               145083 non-null object
coupon_get_cnt           145083 non-null float64
coupon_get_sum           145083 non-null float64
source_company_id        145904 non-null object
source_company_cnt       145083 non-null float64
get_coupon_type_cnt      145083 non-null float64
get_com_roomnum          102154 non-null object
get_com_regioncode       102154 non-null object
get_com_managetype       102154 non-null object
get_com_vodsystemtype    102154 non-null object
get_coupon_crosscom      145904 non-null int64
dtypes: float64(4), int64(1), object(6)
memory usage: 13.4+ MB


In [52]:
len(pd.value_counts(get_coupon['customerid']))

145083

### 去除customerid为空的行

In [56]:
get_coupon = get_coupon[get_coupon['customerid'].notnull()]

In [57]:
get_coupon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145083 entries, 0 to 145082
Data columns (total 11 columns):
customerid               145083 non-null object
coupon_get_cnt           145083 non-null float64
coupon_get_sum           145083 non-null float64
source_company_id        145083 non-null object
source_company_cnt       145083 non-null float64
get_coupon_type_cnt      145083 non-null float64
get_com_roomnum          101333 non-null object
get_com_regioncode       101333 non-null object
get_com_managetype       101333 non-null object
get_com_vodsystemtype    101333 non-null object
get_coupon_crosscom      145083 non-null int64
dtypes: float64(4), int64(1), object(6)
memory usage: 13.3+ MB


In [58]:
df = pd.merge(user_info,get_coupon,how='outer',left_on='customerid',right_on='customerid')

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490276 entries, 0 to 490275
Data columns (total 30 columns):
customerid                      490276 non-null object
registertime                    368275 non-null object
customer_status                 368275 non-null object
openid                          195848 non-null object
wechat_sex                      81738 non-null object
wechat_city                     81738 non-null object
wechat_province                 81738 non-null object
wechat_country                  81738 non-null object
kmid                            107657 non-null object
member_birthday                 15559 non-null float64
km_sex                          15559 non-null float64
accountcash                     15559 non-null object
accountbalance                  15559 non-null object
integralbalance                 15559 non-null object
consumetotal                    15559 non-null object
consumetimes                    15559 non-null object
member_age_cut    

In [60]:
len(pd.value_counts(df['customerid']))

490276

In [61]:
len(pd.value_counts(df['kmid']))

107657

In [62]:
len(pd.value_counts(df['openid']))

195848

In [63]:
df.shape

(490276, 30)

# 3. 和use_coupon表连

In [64]:
sql = 'SELECT * FROM ljl_use_coupon'
use_coupon = pd.read_sql(sql,conn)

In [65]:
use_coupon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37383 entries, 0 to 37382
Data columns (total 6 columns):
customerid              37383 non-null object
use_coupon_sum          37383 non-null float64
use_coupon_order_sum    37383 non-null float64
use_coupon_type_cnt     37383 non-null int64
use_company_id          37383 non-null object
use_company_cnt         37383 non-null int64
dtypes: float64(2), int64(2), object(2)
memory usage: 2.0+ MB


In [66]:
len(pd.value_counts(use_coupon['customerid']))

37383

In [67]:
df1 = pd.merge(df,use_coupon,how='outer',left_on='customerid',right_on='customerid')

In [68]:
df1.shape

(490627, 35)

In [69]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490627 entries, 0 to 490626
Data columns (total 35 columns):
customerid                      490627 non-null object
registertime                    368275 non-null object
customer_status                 368275 non-null object
openid                          195848 non-null object
wechat_sex                      81738 non-null object
wechat_city                     81738 non-null object
wechat_province                 81738 non-null object
wechat_country                  81738 non-null object
kmid                            107657 non-null object
member_birthday                 15559 non-null float64
km_sex                          15559 non-null float64
accountcash                     15559 non-null object
accountbalance                  15559 non-null object
integralbalance                 15559 non-null object
consumetotal                    15559 non-null object
consumetimes                    15559 non-null object
member_age_cut    

In [70]:
len(pd.value_counts(df1['customerid']))

490627

# 4. 和destine表连

In [21]:
sql = 'SELECT * FROM ljl_destine'
destine = pd.read_sql(sql,conn)

In [22]:
destine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1174 entries, 0 to 1173
Data columns (total 43 columns):
customerid                          1174 non-null object
des_cnt                             1174 non-null int64
des_charge_sum                      1174 non-null float64
des_procharge_sum                   1174 non-null float64
des_reduce_sum                      1174 non-null float64
des_realcharge_sum                  1174 non-null float64
des_workday_cnt                     1174 non-null int64
des_weekend_cnt                     1174 non-null int64
des_work_realchargesum              1174 non-null float64
des_weekend_realchargesum           1174 non-null float64
des_charge_mean                     1174 non-null float64
des_realcharge_mean                 1174 non-null float64
des_guestnumber_mean                1174 non-null float64
des_guestnumber_max                 1174 non-null float64
des_guestnumber_min                 1174 non-null float64
des_charge_max                

In [71]:
df2 = pd.merge(df1,destine,how='outer',left_on='customerid',right_on='customerid')

In [72]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490896 entries, 0 to 490895
Data columns (total 77 columns):
customerid                          490896 non-null object
registertime                        368275 non-null object
customer_status                     368275 non-null object
openid                              195848 non-null object
wechat_sex                          81738 non-null object
wechat_city                         81738 non-null object
wechat_province                     81738 non-null object
wechat_country                      81738 non-null object
kmid                                107657 non-null object
member_birthday                     15559 non-null float64
km_sex                              15559 non-null float64
accountcash                         15559 non-null object
accountbalance                      15559 non-null object
integralbalance                     15559 non-null object
consumetotal                        15559 non-null object
consumetimes

In [73]:
len(pd.value_counts(df2['customerid']))

490896

# 5. 和tag_age_from_song表连

In [26]:
sql = 'SELECT * FROM ljl_tag_age_from_song'
tag_age_song = pd.read_sql(sql,conn)

In [27]:
tag_age_song.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6734 entries, 0 to 6733
Data columns (total 6 columns):
customerid       6734 non-null object
songid           6734 non-null object
song_play_cnt    6734 non-null int64
year             2300 non-null float64
age_year         2300 non-null float64
age_lan          6619 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 368.3+ KB


In [76]:
df3 = pd.merge(df2,tag_age_song,how='outer',left_on='customerid',right_on='customerid')

In [77]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497586 entries, 0 to 497585
Data columns (total 82 columns):
customerid                          497586 non-null object
registertime                        368275 non-null object
customer_status                     368275 non-null object
openid                              195848 non-null object
wechat_sex                          81738 non-null object
wechat_city                         81738 non-null object
wechat_province                     81738 non-null object
wechat_country                      81738 non-null object
kmid                                107657 non-null object
member_birthday                     15559 non-null float64
km_sex                              15559 non-null float64
accountcash                         15559 non-null object
accountbalance                      15559 non-null object
integralbalance                     15559 non-null object
consumetotal                        15559 non-null object
consumetimes

In [78]:
len(pd.value_counts(df3['customerid']))

497586

In [79]:
len(pd.value_counts(df3['openid']))

195848

# 6.和shop表连 (用openid)

In [30]:
# 6. 读取shop
sql = 'SELECT * FROM ljl_shop'
shop = pd.read_sql(sql,conn)

In [31]:
shop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49978 entries, 0 to 49977
Data columns (total 32 columns):
openid                    49978 non-null object
shop_real_sum             49978 non-null float64
shop_original_sum         49978 non-null float64
shop_wine_cnt             49978 non-null float64
shop_cigarette_cnt        49978 non-null float64
shop_cnt                  49978 non-null float64
shop_before_dawn_cnt      49978 non-null float64
shop_morning_cnt          49978 non-null float64
shop_afternoon_cnt        49978 non-null float64
shop_night_cnt            49978 non-null float64
shop_weekday_cnt          49978 non-null float64
shop_weekend_cnt          49978 non-null float64
shop_before_dawn_sum      49978 non-null float64
shop_morning_sum          49978 non-null float64
shop_afternoon_sum        49978 non-null float64
shop_night_sum            49978 non-null float64
shop_weekday_sum          49978 non-null float64
shop_weekend_sum          49978 non-null float64
shop_real_

In [80]:
total = pd.merge(df3,shop,how='outer',left_on='openid',right_on='openid')

In [81]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503879 entries, 0 to 503878
Columns: 113 entries, customerid to shop_com_vodsystemtype
dtypes: float64(81), object(32)
memory usage: 438.2+ MB


In [82]:
for col in total.columns.values:
    print(col, total[col].notnull().sum())

(u'customerid', 497586)
(u'registertime', 368275)
(u'customer_status', 368275)
(u'openid', 202141)
(u'wechat_sex', 81738)
(u'wechat_city', 81738)
(u'wechat_province', 81738)
(u'wechat_country', 81738)
(u'kmid', 107657)
(u'member_birthday', 15559)
(u'km_sex', 15559)
(u'accountcash', 15559)
(u'accountbalance', 15559)
(u'integralbalance', 15559)
(u'consumetotal', 15559)
(u'consumetimes', 15559)
(u'member_age_cut', 15559)
(u'lastconsume_diff_create_day', 15559)
(u'lastconsume_diff_create_hour', 15559)
(u'lastconsume_diff_now', 15559)
(u'coupon_get_cnt', 145083)
(u'coupon_get_sum', 145083)
(u'source_company_id', 145083)
(u'source_company_cnt', 145083)
(u'get_coupon_type_cnt', 145083)
(u'get_com_roomnum', 101333)
(u'get_com_regioncode', 101333)
(u'get_com_managetype', 101333)
(u'get_com_vodsystemtype', 101333)
(u'get_coupon_crosscom', 145083)
(u'use_coupon_sum', 37383)
(u'use_coupon_order_sum', 37383)
(u'use_coupon_type_cnt', 37383)
(u'use_company_id', 37383)
(u'use_company_cnt', 37383)
(u'd

In [83]:
len(pd.value_counts(total['openid']))

202141

In [84]:
len(pd.value_counts(total['customerid']))

497586

In [85]:
len(pd.value_counts(total['kmid']))

107657

In [86]:
total['customerid'][total['customerid'].isnull()] = total['openid']

D:\Users\Jialing Liao\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [90]:
for col in total.columns.values:
    print(col, total[col].notnull().sum())

(u'customerid', 503879)
(u'registertime', 368275)
(u'customer_status', 368275)
(u'openid', 202141)
(u'wechat_sex', 81738)
(u'wechat_city', 81738)
(u'wechat_province', 81738)
(u'wechat_country', 81738)
(u'kmid', 107657)
(u'member_birthday', 15559)
(u'km_sex', 15559)
(u'accountcash', 15559)
(u'accountbalance', 15559)
(u'integralbalance', 15559)
(u'consumetotal', 15559)
(u'consumetimes', 15559)
(u'member_age_cut', 15559)
(u'lastconsume_diff_create_day', 15559)
(u'lastconsume_diff_create_hour', 15559)
(u'lastconsume_diff_now', 15559)
(u'coupon_get_cnt', 145083)
(u'coupon_get_sum', 145083)
(u'source_company_id', 145083)
(u'source_company_cnt', 145083)
(u'get_coupon_type_cnt', 145083)
(u'get_com_roomnum', 101333)
(u'get_com_regioncode', 101333)
(u'get_com_managetype', 101333)
(u'get_com_vodsystemtype', 101333)
(u'get_coupon_crosscom', 145083)
(u'use_coupon_sum', 37383)
(u'use_coupon_order_sum', 37383)
(u'use_coupon_type_cnt', 37383)
(u'use_company_id', 37383)
(u'use_company_cnt', 37383)
(u'd

In [91]:
total.shape

(503879, 113)

In [103]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

total.to_csv('G:/user_all.csv')